<a href="https://colab.research.google.com/github/72soniya06/sentiment-analysis/blob/main/Sentiment_Analysis_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [3]:


# Load dataset
df = pd.read_csv("/content/Womens Clothing E-Commerce Reviews.csv.zip")
df = df[['Review Text', 'Rating']].dropna()

# Map Ratings to Sentiment
def label_sentiment(rating):
    if rating >= 4:
        return "positive"
    elif rating == 3:
        return "neutral"
    else:
        return "negative"

df['Sentiment'] = df['Rating'].apply(label_sentiment)


In [4]:


# Remove stopwords and tokenize
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    words = text.lower().split()
    return ' '.join([word for word in words if word not in stop_words])

df['Cleaned_Review'] = df['Review Text'].apply(clean_text)

# Vectorize
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Cleaned_Review'])
y = df['Sentiment']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
model = MultinomialNB()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    negative       0.60      0.44      0.50       457
     neutral       0.44      0.35      0.39       588
    positive       0.89      0.95      0.92      3484

    accuracy                           0.82      4529
   macro avg       0.64      0.58      0.60      4529
weighted avg       0.80      0.82      0.81      4529



In [8]:
# Enter user review
user_review = input("Enter a product review: ")
user_review_clean = clean_text(user_review)
user_vec = vectorizer.transform([user_review_clean])
prediction = model.predict(user_vec)

print(f"\nPredicted Sentiment: {prediction[0]}")


Enter a product review: product is very bad

Predicted Sentiment: negative


In [9]:
!pip install gradio

import gradio as gr

def predict_sentiment(review):
    cleaned = clean_text(review)
    vec = vectorizer.transform([cleaned])
    prediction = model.predict(vec)
    return prediction[0]

interface = gr.Interface(fn=predict_sentiment, inputs="text", outputs="text",
                         title="Product Review Sentiment Analyzer",
                         description="Enter a product review and see if it's positive, negative, or neutral.")

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7bf972126fad2c6328.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
